In [1]:
!hdfs dfsadmin -safemode leave

Safe mode is OFF


In [15]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [25]:
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")

In [26]:
spark = SparkSession.builder.master("yarn").appName("vorkhlik_spark").getOrCreate()

In [5]:
!hdfs dfs -rm -r ml-latest-small
!hdfs dfs -put ml-latest-small .

rm: `ml-latest-small': No such file or directory


In [27]:
from pyspark.sql.types import *

In [28]:
tags_schema = StructType([
    StructField("userId", IntegerType(), True), 
    StructField("movieId", IntegerType(), True), 
    StructField("tag", StringType(), True), 
    StructField("timestamp", LongType(), True)])

ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

In [29]:
%%time
ratings_df = spark\
    .read\
    .format("csv")\
    .option("header", "True")\
    .schema(ratings_schema)\
    .load("ml-latest-small/ratings.csv")

CPU times: user 8.3 ms, sys: 525 µs, total: 8.83 ms
Wall time: 222 ms


In [30]:
%%time
tags_df = spark\
    .read\
    .format("csv")\
    .option("header", "True")\
    .schema(tags_schema)\
    .load("ml-latest-small/tags.csv")

CPU times: user 5.09 ms, sys: 2.09 ms, total: 7.18 ms
Wall time: 83.5 ms


In [31]:
print(tags_df.count(), ratings_df.count())

3683 100836


Для расчета было выполнено 2 стейджа для каждой операции count, в каждом стейдже 1 таска